In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [2]:
# ① 데이터 읽어오기
train_file = os.path.join('./models/intent', "Intent_train_data.csv")
df = pd.read_csv(train_file, delimiter=',', header=None)
df = df[[0, 1, 2]]

In [3]:
df.columns = ['query', 'intent', 'intent_info']
seq_len = list(map(lambda x : len(x.split(' ')), df['query']))
max(seq_len)  # 문장 최대길이

8

In [4]:
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = max(seq_len)

def GlobalParams():
    global MAX_SEQ_LEN

In [5]:
# '질문(query)' 과 '의도(intent)'
queries = df['query'].tolist()
intents = df['intent'].tolist()

In [6]:
queries[:20]

['빵종류?',
 '종류?',
 '음료종류?',
 '커피종류?',
 '식사종류?',
 '에이드종류?',
 '사이드메뉴는 뭐 있어?',
 '사이드 메뉴는 뭐 있어?',
 '빵종류는 뭐 있어?',
 '종류는 뭐 있어?',
 '음료종류는 뭐 있어?',
 '커피종류는 뭐 있어?',
 '식사종류는 뭐 있어?',
 '에이드종류는 뭐 있어?',
 '사이드메뉴는?',
 '사이드 메뉴는?',
 '빵종류는?',
 '종류는?',
 '음료종류는?',
 '커피종류는?']

In [7]:
intents[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
# 전처리 모듈 불러오기
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [9]:
# 단어 시퀀스 생성 (가장 시간 많이 걸림)
# 해당 단어에 매칭되는 번호로 시퀀스 생성

# ★ 시간 제법 걸림 * 몇십초 정도..

sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [10]:
lens = []
for i in sequences:
    lens.append(len(i))
max(lens)

9

In [11]:
# ② 단어 인덱스 시퀀스 벡터 
# 단어 시퀀스 벡터 크기 (MAX_SEQ_LEN 로 동일하게 맞추기, 패딩처리)
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [12]:
padded_seqs

array([[  64, 8941,    0, ...,    0,    0,    0],
       [8941,    0,    0, ...,    0,    0,    0],
       [3297, 8941,    0, ...,    0,    0,    0],
       ...,
       [   1,    3,    2, ...,    0,    0,    0],
       [   1,    3,    2, ...,    0,    0,    0],
       [   1,    3,    2, ...,    0,    0,    0]])

In [13]:
len(p.word_index)

17751

In [14]:
# ③ 학습용, 검증용, 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries), seed=12) # 랜덤 섞기

# 학습셋:검셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 10
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

In [15]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입력크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(10, name='logits')(dropout_hidden)  # 최종적으로 12가의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(10, activation=tf.nn.softmax)(logits)


In [16]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 10, 128)      2272256     ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 10, 128)      0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 8, 128)       49280       ['dropout[0][0]']                
                                                                                              

In [18]:
# 모델학습
# ★ 시간 걸림 ★ 
checkpoint_cb = keras.callbacks.ModelCheckpoint('./models/intent/intent_model.h5', save_best_only=True) # 제일 좋은 모델 저장
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1,
         callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
381/381 [==============================] - 14s 34ms/step - loss: 0.9521 - accuracy: 0.6863 - val_loss: 0.2001 - val_accuracy: 0.9420
Epoch 2/10
381/381 [==============================] - 12s 32ms/step - loss: 0.2428 - accuracy: 0.9233 - val_loss: 0.1672 - val_accuracy: 0.9448
Epoch 3/10
381/381 [==============================] - 12s 31ms/step - loss: 0.1970 - accuracy: 0.9312 - val_loss: 0.1467 - val_accuracy: 0.9475
Epoch 4/10
381/381 [==============================] - 12s 31ms/step - loss: 0.1945 - accuracy: 0.9311 - val_loss: 0.1396 - val_accuracy: 0.9471
Epoch 5/10
381/381 [==============================] - 12s 31ms/step - loss: 0.1668 - accuracy: 0.9412 - val_loss: 0.1444 - val_accuracy: 0.9480
Epoch 6/10
381/381 [==============================] - 12s 31ms/step - loss: 0.1714 - accuracy: 0.9404 - val_loss: 0.1231 - val_accuracy: 0.9489
Epoch 7/10
381/381 [==============================] - 12s 31ms/step - loss: 0.1614 - accuracy: 0.9383 - val_loss: 0.1397 - val_accuracy:

In [19]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

55/55 [==============================] - 0s 2ms/step - loss: 0.1500 - accuracy: 0.9429
Accuracy: 94.290978
loss: 0.149953


## Word2index_Test

In [20]:
from utils.Preprocess import Preprocess
from models.intent.IntentModel import IntentModel
import os

p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [21]:
intent = IntentModel(model_name=os.path.join('./models/intent', 'intent_model.h5'), preprocess=p)

In [164]:
query = "카카오페이 결제" # 필수항목 : 메뉴, 메뉴판, 주문, 결제, 포인트, 할인, 원산지, 텀블러, 영업시간, 테이크아웃, 추천, 화장실, 와이파이, 주문취소, 취소, 아메리카노

# 1번 모델이 우세

# 메뉴판 0 
# 테이크아웃 8
# 영업시간 6
# 원산지 4
# 와이파이 5

In [165]:
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

In [166]:
print(query)
print('의도 예측 클래스 : ', predict)
print('의도 예측 레이블 : ', predict_label)

카카오페이 결제
의도 예측 클래스 :  3
의도 예측 레이블 :  결제, 쿠폰, 포인트
